# Mount colab with google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load Model

In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/AI_Project/Model/my_model_VGG16_accuracy_84")

Poject working path 

In [ ]:
% cd "/content/drive/MyDrive/Colab Notebooks/AI_Project"

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import glob

In [ ]:
IMAGE_SIZE = 224

In [ ]:
img_dir = "/content/drive/MyDrive/Colab Notebooks/AI_Project/Dataset/images" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
files.sort() #We sort the images in alphabetical order to match them to the xml files containing the annotations of the bounding boxes
X=[]
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    X.append(np.array(img))

In [ ]:
from lxml import etree
def resizeannotation(f):
    tree = etree.parse(f)
    for dim in tree.xpath("size"):
        width = int(dim.xpath("width")[0].text)
        height = int(dim.xpath("height")[0].text)
    for dim in tree.xpath("object/bndbox"):
        xmin = int(dim.xpath("xmin")[0].text)/(width/IMAGE_SIZE)
        ymin = int(dim.xpath("ymin")[0].text)/(height/IMAGE_SIZE)
        xmax = int(dim.xpath("xmax")[0].text)/(width/IMAGE_SIZE)
        ymax = int(dim.xpath("ymax")[0].text)/(height/IMAGE_SIZE)
    return [int(xmax), int(ymax), int(xmin), int(ymin)]

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AI_Project/Dataset/annotations'
text_files = ['/content/drive/MyDrive/Colab Notebooks/AI_Project/Dataset/annotations/'+f for f in sorted(os.listdir(path))]
y=[]
for i in text_files:
    y.append(resizeannotation(i))

In [ ]:
plt.figure(figsize=(15,15))
for i in range(0,9) :
    plt.subplot(3,3,i+1)
    plt.axis('off')
    plt.imshow(X[i], cmap = 'jet')

In [ ]:
plt.figure(figsize=(15,15))
for i in range(0,9) :
    plt.subplot(3,3,i+1)
    plt.axis('off')
    image = cv2.rectangle(X[i],(y[i][0],y[i][1]),(y[i][2],y[i][3]),(255,255,0))
    #plt.imshow(X[i], cmap = 'jet')
    plt.imshow(image)

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
#Renormalisation
X = X / 255
y = y / 255

# Train Test Split
To perform model predictions on test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[0:433], y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
y_cnn = model.predict(X_test)

In [ ]:
plt.figure(figsize=(20,40))
for i in range(35,43) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    ny = y_cnn[i]*255
    image = cv2.rectangle(X_test[i],(int(ny[0]),int(ny[1])),(int(ny[2]),int(ny[3])),(0, 255, 0))
    plt.imshow(image)

In [ ]:
X_test.shape